## Testing Cassiopeia Riot API interface

In [273]:
import cassiopeia as cass
import constants
import random
import math

In [274]:
# Set API key from untracked constants.py file
cass.set_riot_api_key(constants.RIOT_API_KEY)  # Must update constants.py every 24 hours with new API key

In [275]:
# Explore config of cassiopeia
cass.get_default_config()

{'global': {'version_from_match': 'patch'},
 'plugins': {},
 'pipeline': {'Cache': {},
  'DDragon': {},
  'RiotAPI': {'api_key': '$RIOT_API_KEY'}},
 'logging': {'print_calls': True,
  'print_riot_api_key': False,
  'default': 'WARNING',
  'core': 'WARNING'}}

## Web-scraping for top League of Legends players

In [276]:
import requests
from bs4 import BeautifulSoup
from bs4 import Comment
import pandas as pd
import re
from urllib.parse import unquote

In [277]:
# For op.gg, need to specify the User-Agent
opgg_url = 'https://www.op.gg/leaderboards/tier?region=kr&type=ladder&page=1'
opgg_response = requests.get(opgg_url)
opgg_response.encoding = 'utf-8'
opgg_html = opgg_response.text
opgg_soup = BeautifulSoup(opgg_html, "html.parser")

In [278]:
# Get number of KR players from op.gg
total_player_div = opgg_soup.find_all('div', class_='css-1a7bwbq ef3mb1o0')
total_player_count = int("".join(re.search(r'(?<=Total )\S+', str(total_player_div)).group().split(',')))
high_elo_count = math.floor(total_player_count * 0.04)  # Get top 4% of players from KR player base
num_pages = math.ceil(high_elo_count / 100)
num_pages

665

In [279]:
# Find player names
user_info_hyperlinks = opgg_soup.find_all('a', class_='summoner-link')

In [280]:
# For testing functionality of cassiopeia
test_list = user_info_hyperlinks[0:5]

In [281]:
def get_usernames_taglines(hyperlink_string):
    """Generates username and tagline for league players

    Args:
        hyperlink_list (list): hyperlinks for players extracted from HTML
    Returns:
        user_info (dict): A dictionary of usernames and taglines
    """
    player_info = re.search(r'(?<=kr\/)\S+(?=">)', str(hyperlink_string)).group()
    # Splits decoded username and tagline
    decoded_username_tagline = unquote(player_info).split('-')
    
    return {decoded_username_tagline[0]:decoded_username_tagline[1]}

In [282]:
# Creates a dictionary of unique player:tagline key:value pairs
player_data_dict = {}
for player in test_list:  # Replace test_list with user_info_hyperlinks in final code
    player_info = get_usernames_taglines(player)
    player_data_dict.update(player_info)

In [ ]:
# Gathers info from account.summoner
for username, tagline in player_data_dict.items():
    account = cass.get_account(name=username, tagline=f'{tagline}', region='KR')
    summoner = account.summoner

Making call: https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/Blue/KR33
Making call: https://ddragon.leagueoflegends.com/realms/kr.json
Making call: https://ddragon.leagueoflegends.com/cdn/15.3.1/data/ko_KR/championFull.json
They enjoy playing champions such as 카르마.
Making call: https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/pupuouo/KR1
They enjoy playing champions such as 모데카이저.
Making call: https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/이%20차가%20식기전에/KR1
They enjoy playing champions such as 신드라.
Making call: https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/斗鱼南波儿丶cl666/0028
They enjoy playing champions such as 나서스.
Making call: https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/스컬지/K%20T
They enjoy playing champions such as 카직스.
